In [9]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [10]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Info'].tolist()

总数量：1


['https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=8566492']

In [11]:
crawler_status = 'error'

crawler_status

'error'

In [12]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [13]:
a = 0

resp = requests.get(input_.loc[a, 'Info'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE HTML>\n<html class="no-js REPLACEHTMLCLASS" onclick="" lang="en">\n<head>\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n<link rel="stylesheet" type="text/css" href="/jslib/combined/css/desktopmoreinfo.css?1717583056">\n\n<script>\nif(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};\n_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift()

In [14]:
from bs4 import BeautifulSoup
from lxml import etree

In [15]:
soup = BeautifulSoup(resp)
html = etree.HTML(str(soup))

with open('./info.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
 <head>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="/jslib/combined/css/desktopmoreinfo.css?1717583056" rel="stylesheet" type="text/css"/>
  <script>
   if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for

In [16]:
html.xpath('//span[@id="vew_partnumber[1]"]/text()')[0].strip() != input_.loc[a, 'Part_Number']

True

In [17]:
# security_code = html.xpath('//div[@class="center"]/div')

# security_code

In [18]:
# len(security_code) != 0 and 'security code' in security_code[0].text

In [19]:
# no_parts_found = html.xpath('//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')

# len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found'

In [20]:
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]

list_tr

[<Element tr at 0x1268aec00>,
 <Element tr at 0x10b87eac0>,
 <Element tr at 0x1265962c0>,
 <Element tr at 0x12696c2c0>,
 <Element tr at 0x12696c3c0>,
 <Element tr at 0x12696c400>,
 <Element tr at 0x12696c440>,
 <Element tr at 0x12696c480>,
 <Element tr at 0x12696c340>,
 <Element tr at 0x12696c4c0>,
 <Element tr at 0x12696c500>,
 <Element tr at 0x12696c540>,
 <Element tr at 0x12696c580>,
 <Element tr at 0x12696c5c0>,
 <Element tr at 0x12696c600>,
 <Element tr at 0x12696c640>,
 <Element tr at 0x12696c680>,
 <Element tr at 0x12696c6c0>,
 <Element tr at 0x12696c700>,
 <Element tr at 0x12696c740>,
 <Element tr at 0x12696c780>,
 <Element tr at 0x12696c7c0>,
 <Element tr at 0x12696c800>,
 <Element tr at 0x12696c840>]

In [21]:
df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

for tr in list_tr:
    df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [22]:
crawler_status = 'ok'

crawler_status

'ok'

In [23]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                             'Info': input_.loc[a, 'Info']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [24]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,Info,clutch included,clutch maximum diameter,coil clock position,coil voltage,compatible refrigerant oil type,compressor type,distance from clutch to mounting flange,ecv harness connector,mounting hardware?,...,o-ring b quantity,oil filled,pulley belt type,pulley diameter mm,pulley groove quantity,pulley lip diameter,pulley ridge diameter,refrigerant type,terminal type,type
0,https://www.rockauto.com/en/moreinfo.php?pt=66...,Included,4.31 IN,1 Wire Lead,12 2H,PAG 46,CVC,1.2 IN,"Compressor SE36F, Vehicle SE36M",Not Included,...,1,Included,Serpentine,109.54 mm,5,4.31 IN,4.31 IN,R134a,Pin,DELPHI


In [25]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
